In [12]:
import argparse
import os
import numpy as np
from tqdm import tqdm
from glob import glob
import cv2
import csv
import json
import collections as cl

from modeling.sync_batchnorm.replicate import patch_replication_callback
from modeling.deeplab import *
from utils.loss import SegmentationLosses
from utils.calculate_weights import calculate_weigths_labels
from utils.lr_scheduler import LR_Scheduler
from utils.saver import Saver
#from utils.summaries import TensorboardSummary
from utils.metrics import Evaluator



## モデルの定義

In [13]:
model = DeepLab(num_classes=5, backbone='mobilenet')

print(model)

DeepLab(
  (backbone): MobileNetV2(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): SynchronizedBatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
          (1): SynchronizedBatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace)
          (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (4): SynchronizedBatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): SynchronizedBatchNorm2d(96, eps=1e-05, momentum=0

## 重みの読み込み

In [14]:
weight_path = "./run/factory/deeplab-mobilenet/experiment_1/checkpoint.pth.tar"

checkpoint = torch.load(weight_path)

model.load_state_dict(checkpoint['state_dict'])

device = torch.device("cuda:0")
model = model.to(device)
model = nn.DataParallel(model)

## 関数の定義

In [15]:
from glob import glob
base_path = "../kataoka/all_frames/20180904/"

p = glob(base_path+"*")


### load_img関数
引数:  画像のpath
処理:  画像の正規化、前処理
返り値:  画像ベクトル

### segmentation関数
引数:  load_imgで前処理した画像ベクトル
処理:  モデルにpredictionさせる
返り値:  one-hot

### plot_seg関数
引数:  画像ベクトル , セグメンテーション
処理:  セグメンテーションした結果を表示する

### convert_segmap関数
引数:  mask
処理:  体の部位ごとにセグメントする
返り値:  セグメンテーションされた画像ベクトル

In [16]:

from PIL import Image
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)

def load_img(p):
    x =  np.array(Image.open('../kataoka/all_frames/' + p).convert('RGB'), dtype=np.float32)
    x /= 255.0
    x -= mean
    x /= std
    x = x.transpose((2, 0, 1))
    #x = torch.from_numpy(x).float()
    #x = x.view(1, 3, 480, 640)
    return x
    
def segmentation(x):

    seg = model(x)

    seg = torch.argmax(seg, dim=1)
    return seg
    

import matplotlib.pyplot as plt
def plot_seg(x,seg):
    x = x.view(3,w,h).transpose(0,1).transpose(1,2).numpy()
    seg = seg.view(w,h).numpy()
    seg = convert_segmap(seg)
    seg = seg.transpose(1,2,0)
    plt.subplot(121)
    plt.imshow(x)
    plt.subplot(122)
    plt.imshow(seg.astype(np.int))
    
def convert_segmap(mask):
    arm = np.zeros(mask.shape)
    head = np.zeros(mask.shape)
    body = np.zeros(mask.shape)
    leg = np.zeros(mask.shape)
    arm = np.where(mask==1, 255, 0)
    head = np.where(mask==2, 255, 0)
    body = np.where(mask==3, 255, 0)
    leg = np.where(mask==4, 255, 0)
    arm += leg
    head += leg
    body += leg
    #足には元から色がついているから返り血として渡す必要がない
    return np.array([arm, head, body])


## 学習させる

In [1]:
!python /home/kataoka/src/pytorch-deeplab-xception/train.py --backbone mobilenet --dataset factory --epochs 250 --batch-size 10

Namespace(backbone='mobilenet', base_size=513, batch_size=10, checkname='deeplab-mobilenet', crop_size=513, cuda=True, dataset='factory', epochs=250, eval_interval=1, freeze_bn=False, ft=False, gpu_ids=[0], loss_type='ce', lr=0.025, lr_scheduler='poly', momentum=0.9, nesterov=False, no_cuda=False, no_val=False, out_stride=16, resume=None, seed=1, start_epoch=0, sync_bn=False, test_batch_size=10, use_balanced_weights=False, use_sbd=True, weight_decay=0.0005, workers=4)
Using poly LR Scheduler!
Starting Epoch: 0
Total Epoches: 250
  0%|                                                    | 0/27 [00:00<?, ?it/s]
=>Epoches 0, learning rate = 0.0250,                 previous best = 0.0000
THCudaCheck FAIL file=/pytorch/aten/src/THC/THCGeneral.cpp line=405 error=11 : invalid argument
/home/kataoka/py36/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
Tr

## 各部位の中心座標を求めcsvファイルに出力

In [17]:
def seg_sentral(seg , order,image_path):
    arm = np.dstack(np.where(seg[order] ==3))
    head = np.dstack(np.where(seg[order] ==2))
    body = np.dstack(np.where(seg[order] ==1))
    leg = np.dstack(np.where(seg[order] ==4))
    
    # xとy座標が逆になっているの注意
    arm = [0 if np.isnan(s) else int(s) for s in arm[0].mean(axis=0)]
    head = [0 if np.isnan(s) else int(s) for s in head[0].mean(axis=0)]
    body = [0 if np.isnan(s) else int(s) for s in body[0].mean(axis=0)]
    leg = [0 if np.isnan(s)  else int(s) for s in leg[0].mean(axis=0)]
    
    #逆を戻す
    arm.reverse()
    head.reverse()
    body.reverse()
    leg.reverse()

    return [image_path,arm,head,body,leg]

def output_seg_sentral(seg , order , image):
    body = np.dstack(np.where(seg[order] ==3))
    head = np.dstack(np.where(seg[order] ==2))
    arm = np.dstack(np.where(seg[order] ==1))
    leg = np.dstack(np.where(seg[order] ==4))
    
    arm = [0 if np.isnan(s) else int(s) for s in arm[0].mean(axis=0)]
    head = [0 if np.isnan(s) else int(s) for s in head[0].mean(axis=0)]
    body = [0 if np.isnan(s) else int(s) for s in body[0].mean(axis=0)]
    leg = [0 if np.isnan(s)  else int(s) for s in leg[0].mean(axis=0)]
    
    image = cv2.circle(image, (arm[1],arm[0]), 10 , (255, 255, 0), -1)
    image = cv2.putText(image, 'arm', (550,110), cv2.FONT_HERSHEY_PLAIN, 1.2, (255,255,0), 2, cv2.LINE_AA)
    image = cv2.circle(image, (head[1],head[0]), 10 , (255, 0, 255), -1)
    image = cv2.putText(image, 'head', (550,50), cv2.FONT_HERSHEY_PLAIN, 1.2, (255,0,255), 2, cv2.LINE_AA)
    image = cv2.circle(image, (body[1],body[0]), 10 , (255, 255, 255), -1)
    image = cv2.putText(image, 'body', (550,80), cv2.FONT_HERSHEY_PLAIN, 1.2, (255,255,255), 2, cv2.LINE_AA)
    image = cv2.circle(image, (leg[1],leg[0]), 10 , (128, 0, 0), -1)
    image = cv2.putText(image, 'leg', (550,140), cv2.FONT_HERSHEY_PLAIN, 1.2, (128,0,0), 2, cv2.LINE_AA)
    
    return image

In [18]:
csv_file = open("data/0904seg_image_path.csv", "r")
f = csv.reader(csv_file)


In [19]:
all_video_paths = []
for item in f:
    all_video_paths.extend(item)
    
def check_direct(path):
    if not os.path.exists(path):
        os.mkdir(path)
    

In [23]:
# セグメンテーションした結果をJSONの形式にフォーマット(教師データの形にする)
model.eval()
test = 0
data_dict = cl.OrderedDict()
# f = open('data/0903central_coordinate.json', 'w')
batch_size =0 
while batch_size +10 < len(all_video_paths):
    image_path = all_video_paths[batch_size:batch_size+10]
    im = np.array([load_img(frame_path) for frame_path in all_video_paths[batch_size:batch_size+10]])
    batch_size += 10
    x = torch.from_numpy(im).float().to(device)
    seg = segmentation(x)
    seg = seg.cpu().numpy()
    for image_order in range(seg.shape[0]):
        test = image_path[image_order]
        image = convert_segmap(seg[image_order]).transpose(1,2,0)
        key = image_path[image_order].strip('./kataoka/all_frames/')
#         data_dict[key] = seg[image_order].tolist()
        #writer.writerow(seg_sentral(seg, image_order , image_path[image_order]))
#             test_resut =  output_seg_sentral(seg,image_order,image)
        image_path_list = image_path[image_order].split('/')
        if len(image_path_list) == 4:
            image_path_list.pop(0)
        direct_path = '../kataoka/segmentations/'+ image_path_list[0]
        check_direct(direct_path)
        direct_path = '../kataoka/segmentations/'+ image_path_list[0]+ '/' +  image_path_list[1]
        check_direct(direct_path)
        save_path = direct_path +  '/' + image_path_list[2].replace('_','')
        cv2.imwrite(save_path, image)
# json.dump(data_dict,f)
#f.close()
        

In [21]:
!pwd

/home/jovyan/share/ito


In [15]:
test = cl.OrderedDict()
test["test"] = 0
test["test2"] = 3

In [16]:
json.dump(test,f)

In [10]:
test

OrderedDict([('test', 0), ('test2', 3)])

In [19]:
f = open('data/test.json', 'r')
d  = json.load(f)

JSONDecodeError: Extra data: line 1 column 12 (char 11)